In [1]:
def add_dff_stats_to_specimen_table(session):
    '''
    merges statistics about df/f to the cell specimen table
    operates inplace
    returns the table with additional columns
    '''
    dff_stats = []
    for csid,row in session.cell_specimen_table.iterrows():
        dff = session.dff_traces.loc[csid]['dff']
        dff_stats_single = pd.Series(dff).describe().to_dict()
        dff_stats_single.update({
            'cell_specimen_id': csid,
        })
        dff_stats.append(dff_stats_single)
    dff_stats = pd.DataFrame(dff_stats).set_index('cell_specimen_id')
    df = session.cell_specimen_table.merge(
        dff_stats,
        left_index = True,
        right_index = True
    )
    return df

In [107]:
import visual_behavior.data_access.loading as loading
import visual_behavior.data_access.processing as processing
import visual_behavior.database as db
Database = db.Database
import pandas as pd
pd.set_option('display.max_columns', 500)
oeid = 945586458
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
%%time
session = loading.get_ophys_dataset(oeid)

loading data for 945586458_453989_190912_VISl_0_Vip-IRES-Cre_MESO.1_behavior_225
CPU times: user 12.3 ms, sys: 5.62 ms, total: 18 ms
Wall time: 32.9 ms


In [14]:
%%time
df = add_dff_stats_to_specimen_table(session)

CPU times: user 91 ms, sys: 5.83 ms, total: 96.9 ms
Wall time: 92.9 ms


In [13]:
df

,cell_roi_id,height,image_mask,mask_image_plane,max_correction_down,max_correction_left,max_correction_right,max_correction_up,valid_roi,width,x,y,roi_mask,count,mean,std,min,25%,50%,75%,max
cell_specimen_id,,,,,,,,,,,,,,,,,,,,,
1065772528,945636517,13,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,16,389,357,"[[False, False, False, False, False, False, Fa...",48282.0,0.063956,0.412197,-1.653297,-0.173886,0.007973,0.230774,9.511762
1065772656,945636521,19,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,16,322,330,"[[False, False, False, False, False, False, Fa...",48282.0,0.071712,0.359478,-0.814048,-0.126543,0.003989,0.171768,9.575090
1065771819,945636526,17,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,22,430,316,"[[False, False, False, False, False, False, Fa...",48282.0,0.110052,0.455931,-1.160410,-0.105856,0.012367,0.172936,7.367701
1065771760,945636531,15,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,12,40,312,"[[False, False, False, False, False, False, Fa...",48282.0,0.094587,0.562028,-2.301327,-0.245160,0.011024,0.326021,7.447715
1065771576,945636538,17,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,21,107,278,"[[False, False, False, False, False, False, Fa...",48282.0,0.042915,0.230471,-0.666957,-0.080657,0.001713,0.104918,3.128950
1065772755,945636541,16,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,20,362,267,"[[False, False, False, False, False, False, Fa...",48282.0,0.036931,0.205467,-0.536752,-0.080289,0.002032,0.102364,2.733393
1065771176,945636546,17,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,17,421,243,"[[False, False, False, False, False, False, Fa...",48282.0,0.011874,0.126323,-0.474189,-0.069370,0.000000,0.078240,2.025595
1065772353,945636548,13,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,17,443,393,"[[False, False, False, False, False, False, Fa...",48282.0,0.103811,0.524724,-1.990314,-0.166336,0.014970,0.239731,12.343264
1065772437,945636550,16,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,13,56,379,"[[False, False, False, False, False, False, Fa...",48282.0,0.071065,0.287864,-0.993455,-0.083247,0.007934,0.137232,3.356736


In [80]:
from importlib import reload
import visual_behavior.database as db
reload(db)

<module 'visual_behavior.database' from '/allen/programs/braintv/workgroups/nc-ophys/Doug/code/visual_behavior_analysis/visual_behavior/database.py'>

In [61]:
records = df.drop(columns=['image_mask','roi_mask']).reset_index().to_dict(orient='records')
record = records[0]
# record['cell_specimen_id'] = 12345
for record in records:
    db.log_cell_dff_data(record)

In [50]:

    db_conn = Database('visual_behavior_data')
    collection = db_conn['ophys_data']['dff_summary']
    existing_record = collection.find_one({'cell_roi_id':record['cell_roi_id']})

    # if the cell specimen_id doesn't match what was in the record, log old ID to a list called 'previous_cell_specimen_ids'
    if existing_record and record['cell_specimen_id'] != existing_record['cell_specimen_id']:
        if 'previous_cell_specimen_ids' in existing_record.keys():
            previous_cell_specimen_ids = existing_record['previous_cell_specimen_ids']
            previous_cell_specimen_ids.append(existing_record['cell_specimen_id'])
        else:
            previous_cell_specimen_ids = [existing_record['cell_specimen_id']]
    else:
        previous_cell_specimen_ids = []

    # write record to database    
    record['previous_cell_specimen_ids'] = previous_cell_specimen_ids
    db.update_or_create(collection, record, keys_to_check = ['cell_roi_id'])
    db_conn.close()

{'_id': ObjectId('600f0f0e421350748f21d17e'), 'cell_specimen_id': 12345, 'cell_roi_id': 945636517, 'height': 13, 'mask_image_plane': 0, 'max_correction_down': 13.0373, 'max_correction_left': 8.58747, 'max_correction_right': 7.97365, 'max_correction_up': 2.99062, 'valid_roi': 1, 'width': 16, 'x': 389, 'y': 357, 'count': 48282.0, 'mean': 0.06395630151463524, 'std': 0.4121966938204358, 'min': -1.6532970215812164, '25%': -0.17388646854623596, '50%': 0.007973365295921055, '75%': 0.23077402862473145, 'max': 9.511761695629433, 'previous_cell_specimen_ids': [1065772528]}


In [47]:
record

{'cell_specimen_id': 12345,
 'cell_roi_id': 945636517,
 'height': 13,
 'mask_image_plane': 0,
 'max_correction_down': 13.0373,
 'max_correction_left': 8.58747,
 'max_correction_right': 7.97365,
 'max_correction_up': 2.99062,
 'valid_roi': True,
 'width': 16,
 'x': 389,
 'y': 357,
 'count': 48282.0,
 'mean': 0.06395630151463524,
 'std': 0.4121966938204358,
 'min': -1.6532970215812164,
 '25%': -0.17388646854623596,
 '50%': 0.007973365295921055,
 '75%': 0.23077402862473145,
 'max': 9.511761695629433,
 'previous_cell_specimen_ids': [1065772528]}

In [66]:
ophys_experiment_table = loading.get_filtered_ophys_experiment_table()

In [67]:
ophys_experiment_table.index

Int64Index([953659743, 953659752, 953659749, 953659745, 958527477, 958527474,
            958527471, 958527481, 958527479, 958527464,
            ...
            916220441, 916220443, 916220450, 916220452, 929655730, 929655720,
            929655732, 929655722, 929655737, 929655728],
           dtype='int64', name='ophys_experiment_id', length=1714)

In [73]:
session = loading.get_ophys_dataset(914107611)

loading data for 914107611_453911_190729_VISl_0_Slc17a7-IRES2-Cre_MESO.1_behavior_375


In [74]:
len(session.cell_specimen_table)

64

In [75]:
session.cell_specimen_table

,cell_roi_id,height,image_mask,mask_image_plane,max_correction_down,max_correction_left,max_correction_right,max_correction_up,valid_roi,width,x,y,roi_mask
cell_specimen_id,,,,,,,,,,,,,
NaN,914239624,20,"[[False, False, False, False, False, False, Fa...",1,22.6138,7.10855,18.8385,3.99251,True,29,282,50,"[[False, False, False, False, False, False, Fa..."
NaN,914239628,17,"[[False, False, False, False, False, False, Fa...",1,22.6138,7.10855,18.8385,3.99251,True,27,122,29,"[[False, False, False, False, False, False, Fa..."
NaN,914239636,20,"[[False, False, False, False, False, False, Fa...",1,22.6138,7.10855,18.8385,3.99251,True,23,72,43,"[[False, False, False, False, False, False, Fa..."
NaN,914239653,19,"[[False, False, False, False, False, False, Fa...",0,22.6138,7.10855,18.8385,3.99251,True,23,315,336,"[[False, False, False, False, False, False, Fa..."
NaN,914239661,18,"[[False, False, False, False, False, False, Fa...",0,22.6138,7.10855,18.8385,3.99251,True,17,249,324,"[[False, False, False, False, False, False, Fa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,914239985,26,"[[False, False, False, False, False, False, Fa...",0,22.6138,7.10855,18.8385,3.99251,True,26,155,253,"[[False, False, False, False, False, False, Fa..."
NaN,914239997,20,"[[False, False, False, False, False, False, Fa...",0,22.6138,7.10855,18.8385,3.99251,True,19,235,209,"[[False, False, False, False, False, False, Fa..."
NaN,914240001,21,"[[False, False, False, False, False, False, Fa...",0,22.6138,7.10855,18.8385,3.99251,True,23,430,249,"[[False, False, False, False, False, False, Fa..."


In [78]:
%%time
df = add_dff_stats_to_specimen_table(session)


CPU times: user 6.96 s, sys: 0 ns, total: 6.96 s
Wall time: 6.96 s


In [79]:
df

,cell_roi_id,height,image_mask,mask_image_plane,max_correction_down,max_correction_left,max_correction_right,max_correction_up,valid_roi,width,x,y,roi_mask,count,unique,top,freq
cell_specimen_id,,,,,,,,,,,,,,,,,
NaN,914239624,20,"[[False, False, False, False, False, False, Fa...",1,22.6138,7.10855,18.8385,3.99251,True,29,282,50,"[[False, False, False, False, False, False, Fa...",64,64,"[0.10244548298394426, 0.17395992561342036, 0.0...",1
NaN,914239624,20,"[[False, False, False, False, False, False, Fa...",1,22.6138,7.10855,18.8385,3.99251,True,29,282,50,"[[False, False, False, False, False, False, Fa...",64,64,"[0.10244548298394426, 0.17395992561342036, 0.0...",1
NaN,914239624,20,"[[False, False, False, False, False, False, Fa...",1,22.6138,7.10855,18.8385,3.99251,True,29,282,50,"[[False, False, False, False, False, False, Fa...",64,64,"[0.10244548298394426, 0.17395992561342036, 0.0...",1
NaN,914239624,20,"[[False, False, False, False, False, False, Fa...",1,22.6138,7.10855,18.8385,3.99251,True,29,282,50,"[[False, False, False, False, False, False, Fa...",64,64,"[0.10244548298394426, 0.17395992561342036, 0.0...",1
NaN,914239624,20,"[[False, False, False, False, False, False, Fa...",1,22.6138,7.10855,18.8385,3.99251,True,29,282,50,"[[False, False, False, False, False, False, Fa...",64,64,"[0.10244548298394426, 0.17395992561342036, 0.0...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,914240051,23,"[[False, False, False, False, False, False, Fa...",2,22.6138,7.10855,18.8385,3.99251,True,25,273,271,"[[False, False, False, False, False, False, Fa...",64,64,"[0.10244548298394426, 0.17395992561342036, 0.0...",1
NaN,914240051,23,"[[False, False, False, False, False, False, Fa...",2,22.6138,7.10855,18.8385,3.99251,True,25,273,271,"[[False, False, False, False, False, False, Fa...",64,64,"[0.10244548298394426, 0.17395992561342036, 0.0...",1
NaN,914240051,23,"[[False, False, False, False, False, False, Fa...",2,22.6138,7.10855,18.8385,3.99251,True,25,273,271,"[[False, False, False, False, False, False, Fa...",64,64,"[0.10244548298394426, 0.17395992561342036, 0.0...",1


In [189]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [201]:
import visual_behavior.database as db
dff_summary = db.get_cell_dff_data()
dff_summary

,_id,index,cell_specimen_id,cell_roi_id,height,mask_image_plane,max_correction_down,max_correction_left,max_correction_right,max_correction_up,valid_roi,width,x,y,count,mean,std,min,25%,50%,75%,max,ophys_experiment_id,previous_cell_specimen_ids
0,601058ee10804f2091d0ccd7,0,1.065628e+09,953812964,17,0,5.18794,0.807048,9.15835,6.61726,1,18,212,352,48332.0,0.294806,1.805512,-5.347699,-0.361722,0.014080,0.463651,48.394604,953659749,[]
1,601058ee10804f2091d0ccd9,1,1.065628e+09,953812966,16,0,5.18794,0.807048,9.15835,6.61726,1,17,247,392,48332.0,0.025899,0.167097,-0.466131,-0.075171,0.001319,0.092652,2.078784,953659749,[]
2,601058ee10804f2091d0ccdb,2,1.065627e+09,953812968,14,0,5.18794,0.807048,9.15835,6.61726,1,22,157,256,48332.0,0.023825,0.155572,-0.479241,-0.063716,0.002201,0.076535,1.926400,953659749,[]
3,601058ef10804f2091d0ccdd,3,1.065627e+09,953812971,14,0,5.18794,0.807048,9.15835,6.61726,1,16,107,290,48332.0,0.043657,0.227389,-0.632441,-0.081819,0.004369,0.109638,4.221920,953659749,[]
4,601058ef10804f2091d0ccdf,4,1.065628e+09,953812973,22,1,5.18794,0.807048,9.15835,6.61726,1,19,208,332,48332.0,0.042960,0.226614,-0.500088,-0.066962,0.003303,0.089539,3.985577,953659749,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99231,60105ec8e5e7064a4c9b2e9d,44,1.065772e+09,1039433991,16,0,8.02597,4.912970,14.38020,7.63704,1,33,266,13,48129.0,0.004612,0.092358,-0.289746,-0.052982,0.000000,0.056050,1.818985,1039384490,[]
99232,60105ec8e5e7064a4c9b2e9f,45,1.065773e+09,1039434002,14,0,8.02597,4.912970,14.38020,7.63704,1,22,394,37,48129.0,0.007546,0.115538,-0.356354,-0.067794,0.000000,0.073477,2.045214,1039384490,[]
99233,60105ec8e5e7064a4c9b2ea1,46,1.065773e+09,1039434006,16,0,8.02597,4.912970,14.38020,7.63704,1,18,360,37,48129.0,0.006086,0.113892,-0.367381,-0.065933,0.000000,0.069332,1.568705,1039384490,[]
99234,60105ec8e5e7064a4c9b2ea3,47,1.065773e+09,1039434017,19,0,8.02597,4.912970,14.38020,7.63704,1,21,68,325,48129.0,0.009499,0.117376,-0.417319,-0.060134,0.000000,0.065433,2.025821,1039384490,[]


In [202]:
len(dff_summary['cell_specimen_id'].unique())

37063

In [203]:
len(dff_summary['cell_roi_id'].unique())

99236

In [204]:
len(dff_summary[pd.isnull(dff_summary['cell_specimen_id'])])

6724

In [205]:
len(dff_summary[pd.isnull(dff_summary['cell_specimen_id'])]['ophys_experiment_id'].unique())

171

In [212]:
nan_ids = list(dff_summary[pd.isnull(dff_summary['cell_specimen_id'])]['ophys_experiment_id'].unique())

In [231]:
for ii, nan_id in enumerate(nan_ids):
    all_cells_this_id = dff_summary.query('ophys_experiment_id == @nan_id')
    print('{}\tfor OEID {} - {} CSIDs are NaN, {} CSIDs are not NaN'.format(
        ii, 
        nan_id, 
        pd.isnull(all_cells_this_id['cell_specimen_id']).sum(), 
        pd.notnull(all_cells_this_id['cell_specimen_id']).sum()
    ))

0	for OEID 1039693317 - 60 CSIDs are NaN, 0 CSIDs are not NaN
1	for OEID 1039954540 - 56 CSIDs are NaN, 0 CSIDs are not NaN
2	for OEID 1039954544 - 2 CSIDs are NaN, 0 CSIDs are not NaN
3	for OEID 909177759 - 40 CSIDs are NaN, 0 CSIDs are not NaN
4	for OEID 908381694 - 60 CSIDs are NaN, 0 CSIDs are not NaN
5	for OEID 912791343 - 115 CSIDs are NaN, 0 CSIDs are not NaN
6	for OEID 914107595 - 61 CSIDs are NaN, 0 CSIDs are not NaN
7	for OEID 914107611 - 64 CSIDs are NaN, 0 CSIDs are not NaN
8	for OEID 1040982833 - 1 CSIDs are NaN, 0 CSIDs are not NaN
9	for OEID 1040982825 - 35 CSIDs are NaN, 0 CSIDs are not NaN
10	for OEID 904363938 - 66 CSIDs are NaN, 0 CSIDs are not NaN
11	for OEID 904363928 - 70 CSIDs are NaN, 0 CSIDs are not NaN
12	for OEID 1041171734 - 61 CSIDs are NaN, 0 CSIDs are not NaN
13	for OEID 1041171738 - 1 CSIDs are NaN, 0 CSIDs are not NaN
14	for OEID 783927872 - 216 CSIDs are NaN, 0 CSIDs are not NaN
15	for OEID 789359614 - 151 CSIDs are NaN, 0 CSIDs are not NaN
16	for OEID

In [229]:
pd.isnull(all_cells_this_id['cell_specimen_id']).sum()

60

In [211]:
et = loading.get_filtered_ophys_experiment_table().reset_index()

In [215]:
et.query('ophys_experiment_id in @nan_ids')['project_code'].value_counts()

VisualBehaviorMultiscope             116
VisualBehavior                        29
VisualBehaviorTask1B                  13
VisualBehaviorMultiscope4areasx2d     13
Name: project_code, dtype: int64

In [217]:
et.query('ophys_experiment_id in @nan_ids')['date_of_acquisition'].min()

'2018-11-08 18:38:05.000000'

In [218]:
et.query('ophys_experiment_id in @nan_ids')['date_of_acquisition'].max()

'2020-12-22 11:53:08.107826'

In [232]:
et.query('ophys_experiment_id in @nan_ids')['container_id'].unique()

array([1046576772, 1046576778, 1046576775, 1026722867, 1026722861,
       1026722854, 1026722873, 1026722864, 1064333322,  782536745,
       1052096563, 1052096554, 1052096560, 1052096551, 1052096557,
       1069458233, 1069458250, 1069458236, 1058209392, 1059210985,
        812858755, 1018027781, 1061181340, 1054675423, 1037672801,
       1037672792,  814796558, 1018027896, 1018027884])

In [226]:
et.query('ophys_experiment_id in @nan_ids')['container_workflow_state'].value_counts()

postprocessing    73
completed         63
holding           35
Name: container_workflow_state, dtype: int64

In [162]:
oeid = 945586458
session = loading.get_ophys_dataset(oeid)
session.cell_specimen_table

loading data for 945586458_453989_190912_VISl_0_Vip-IRES-Cre_MESO.1_behavior_225


,cell_roi_id,height,image_mask,mask_image_plane,max_correction_down,max_correction_left,max_correction_right,max_correction_up,valid_roi,width,x,y,roi_mask
cell_specimen_id,,,,,,,,,,,,,
1065772528,945636517,13,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,16,389,357,"[[False, False, False, False, False, False, Fa..."
1065772656,945636521,19,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,16,322,330,"[[False, False, False, False, False, False, Fa..."
1065771819,945636526,17,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,22,430,316,"[[False, False, False, False, False, False, Fa..."
1065771760,945636531,15,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,12,40,312,"[[False, False, False, False, False, False, Fa..."
1065771576,945636538,17,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,21,107,278,"[[False, False, False, False, False, False, Fa..."
1065772755,945636541,16,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,20,362,267,"[[False, False, False, False, False, False, Fa..."
1065771176,945636546,17,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,17,421,243,"[[False, False, False, False, False, False, Fa..."
1065772353,945636548,13,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,17,443,393,"[[False, False, False, False, False, False, Fa..."
1065772437,945636550,16,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,13,56,379,"[[False, False, False, False, False, False, Fa..."


In [167]:
dff_stats = []
for idx,row in session.cell_specimen_table.reset_index().iterrows():
    roi_id = row['cell_roi_id']
    dff = session.dff_traces.query('cell_roi_id == @roi_id')['dff'].iloc[0]
    dff_stats_single = pd.Series(dff).describe().to_dict()
    dff_stats_single.update({
        'cell_roi_id': roi_id
    })
    dff_stats.append(dff_stats_single)
dff_stats = pd.DataFrame(dff_stats)
df = session.cell_specimen_table.reset_index().merge(
    dff_stats,
    left_on = 'cell_roi_id',
    right_on = 'cell_roi_id'
)
df

,cell_specimen_id,cell_roi_id,height,image_mask,mask_image_plane,max_correction_down,max_correction_left,max_correction_right,max_correction_up,valid_roi,width,x,y,roi_mask,count,mean,std,min,25%,50%,75%,max
0,1065772528,945636517,13,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,16,389,357,"[[False, False, False, False, False, False, Fa...",48282.0,0.063956,0.412197,-1.653297,-0.173886,0.007973,0.230774,9.511762
1,1065772656,945636521,19,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,16,322,330,"[[False, False, False, False, False, False, Fa...",48282.0,0.071712,0.359478,-0.814048,-0.126543,0.003989,0.171768,9.575090
2,1065771819,945636526,17,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,22,430,316,"[[False, False, False, False, False, False, Fa...",48282.0,0.110052,0.455931,-1.160410,-0.105856,0.012367,0.172936,7.367701
3,1065771760,945636531,15,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,12,40,312,"[[False, False, False, False, False, False, Fa...",48282.0,0.094587,0.562028,-2.301327,-0.245160,0.011024,0.326021,7.447715
4,1065771576,945636538,17,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,21,107,278,"[[False, False, False, False, False, False, Fa...",48282.0,0.042915,0.230471,-0.666957,-0.080657,0.001713,0.104918,3.128950
5,1065772755,945636541,16,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,20,362,267,"[[False, False, False, False, False, False, Fa...",48282.0,0.036931,0.205467,-0.536752,-0.080289,0.002032,0.102364,2.733393
6,1065771176,945636546,17,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,17,421,243,"[[False, False, False, False, False, False, Fa...",48282.0,0.011874,0.126323,-0.474189,-0.069370,0.000000,0.078240,2.025595
7,1065772353,945636548,13,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,17,443,393,"[[False, False, False, False, False, False, Fa...",48282.0,0.103811,0.524724,-1.990314,-0.166336,0.014970,0.239731,12.343264
8,1065772437,945636550,16,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,13,56,379,"[[False, False, False, False, False, False, Fa...",48282.0,0.071065,0.287864,-0.993455,-0.083247,0.007934,0.137232,3.356736
9,1065772104,945636552,19,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,22,214,76,"[[False, False, False, False, False, False, Fa...",48282.0,0.040154,0.288576,-0.779605,-0.094138,0.003237,0.121524,8.806578


In [171]:
%%time
df = processing.add_dff_stats_to_specimen_table(session)

CPU times: user 89.7 ms, sys: 10.6 ms, total: 100 ms
Wall time: 98.6 ms


In [172]:
df

,cell_roi_id,height,image_mask,mask_image_plane,max_correction_down,max_correction_left,max_correction_right,max_correction_up,valid_roi,width,x,y,roi_mask,count,mean,std,min,25%,50%,75%,max,cell_specimen_id
0,945636517,13,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,16,389,357,"[[False, False, False, False, False, False, Fa...",48282.0,0.063956,0.412197,-1.653297,-0.173886,0.007973,0.230774,9.511762,None
1,945636521,19,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,16,322,330,"[[False, False, False, False, False, False, Fa...",48282.0,0.071712,0.359478,-0.814048,-0.126543,0.003989,0.171768,9.575090,None
2,945636526,17,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,22,430,316,"[[False, False, False, False, False, False, Fa...",48282.0,0.110052,0.455931,-1.160410,-0.105856,0.012367,0.172936,7.367701,None
3,945636531,15,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,12,40,312,"[[False, False, False, False, False, False, Fa...",48282.0,0.094587,0.562028,-2.301327,-0.245160,0.011024,0.326021,7.447715,None
4,945636538,17,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,21,107,278,"[[False, False, False, False, False, False, Fa...",48282.0,0.042915,0.230471,-0.666957,-0.080657,0.001713,0.104918,3.128950,None
5,945636541,16,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,20,362,267,"[[False, False, False, False, False, False, Fa...",48282.0,0.036931,0.205467,-0.536752,-0.080289,0.002032,0.102364,2.733393,None
6,945636546,17,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,17,421,243,"[[False, False, False, False, False, False, Fa...",48282.0,0.011874,0.126323,-0.474189,-0.069370,0.000000,0.078240,2.025595,None
7,945636548,13,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,17,443,393,"[[False, False, False, False, False, False, Fa...",48282.0,0.103811,0.524724,-1.990314,-0.166336,0.014970,0.239731,12.343264,None
8,945636550,16,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,13,56,379,"[[False, False, False, False, False, False, Fa...",48282.0,0.071065,0.287864,-0.993455,-0.083247,0.007934,0.137232,3.356736,None
9,945636552,19,"[[False, False, False, False, False, False, Fa...",0,13.0373,8.58747,7.97365,2.99062,True,22,214,76,"[[False, False, False, False, False, False, Fa...",48282.0,0.040154,0.288576,-0.779605,-0.094138,0.003237,0.121524,8.806578,None


In [120]:
session.dff_traces.loc[csid]['dff']

cell_specimen_id
NaN    [0.645308416409247, 0.48970003142999724, 0.392...
NaN    [0.794563150254804, 0.610184204090928, 0.38921...
NaN    [0.43322508005772375, 0.22731541485014264, 0.1...
NaN    [0.38234114481343123, 0.11372254421102544, 0.2...
NaN    [0.4950174690188693, 0.5622507158901894, 0.534...
                             ...                        
NaN    [0.8568127902455215, 0.5887026517117965, 0.378...
NaN    [0.4058558199750117, 0.30345118051439424, 0.14...
NaN    [0.41786249424105865, 0.4443100710008976, 0.41...
NaN    [1.1457024915346787, 0.3533040852932293, 1.023...
NaN    [0.4998046177469537, 0.39436595223221216, 0.17...
Name: dff, Length: 138, dtype: object

In [124]:
session.cell_specimen_table.reset_index()

,cell_specimen_id,cell_roi_id,height,image_mask,mask_image_plane,max_correction_down,max_correction_left,max_correction_right,max_correction_up,valid_roi,width,x,y,roi_mask
0,None,916314973,20,"[[False, False, False, False, False, False, Fa...",1,11.7604,2.58141,10.5914,6.13556,True,27,27,24,"[[False, False, False, False, False, False, Fa..."
1,None,916314989,27,"[[False, False, False, False, False, False, Fa...",1,11.7604,2.58141,10.5914,6.13556,True,36,299,27,"[[False, False, False, False, False, False, Fa..."
2,None,916315005,14,"[[False, False, False, False, False, False, Fa...",1,11.7604,2.58141,10.5914,6.13556,True,29,186,13,"[[False, False, False, False, False, False, Fa..."
3,None,916315099,20,"[[False, False, False, False, False, False, Fa...",0,11.7604,2.58141,10.5914,6.13556,True,21,373,221,"[[False, False, False, False, False, False, Fa..."
4,None,916315111,19,"[[False, False, False, False, False, False, Fa...",0,11.7604,2.58141,10.5914,6.13556,True,23,230,222,"[[False, False, False, False, False, False, Fa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,None,916316198,27,"[[False, False, False, False, False, False, Fa...",0,11.7604,2.58141,10.5914,6.13556,True,25,103,409,"[[False, False, False, False, False, False, Fa..."
134,None,916316200,25,"[[False, False, False, False, False, False, Fa...",0,11.7604,2.58141,10.5914,6.13556,True,27,270,419,"[[False, False, False, False, False, False, Fa..."
135,None,916316207,21,"[[False, False, False, False, False, False, Fa...",2,11.7604,2.58141,10.5914,6.13556,True,24,387,156,"[[False, False, False, False, False, False, Fa..."
136,None,916316211,14,"[[False, False, False, False, False, False, Fa...",2,11.7604,2.58141,10.5914,6.13556,True,28,333,33,"[[False, False, False, False, False, False, Fa..."


In [155]:
oeid = 916220452

In [173]:
db_conn = Database('visual_behavior_data')
collection = db_conn['ophys_data']['dff_summary']
res = pd.DataFrame(list(collection.find({})))
db_conn.close()

In [176]:
len(res['cell_roi_id'].unique())

96797

In [170]:
db_conn = Database('visual_behavior_data')
collection = db_conn['ophys_data']['dff_summary']
res = collection.delete_many({})
db_conn.close()

In [195]:
session.cell_specimen_table?